In [4]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 25.4 MB/s 


In [5]:
pip install sklearn-evaluation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 4.8 MB/s 
     |████████████████████████████████| 1.5 MB 61.0 MB/s 
     |████████████████████████████████| 843 kB 61.3 MB/s 
     |████████████████████████████████| 96 kB 6.7 MB/s 
     |████████████████████████████████| 1.6 MB 58.8 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [6]:

import numpy as np

# Data utilities
from pandas import read_csv
import matplotlib.pyplot as plt
import os.path
import torch
import h5py

# Machine learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import cross_val_score

In [ ]:
#!wget https://raw.githubusercontent.com/33220311/Extremophiles/main/Embeddings/Sannotations.csv
#!wget https://raw.githubusercontent.com/33220311/Extremophiles/main/Embeddings/SuniqueAnnotations.csv
#!wget https://raw.githubusercontent.com/33220311/Extremophiles/main/Embeddings/Sannotations1024.csv
#!wget https://raw.githubusercontent.com/33220311/Extremophiles/main/Embeddings/SuniqueAnnotations1024.csv

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Utility Function

In [28]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [29]:
# Utility function: Display model score(Loss & Accuracy) across all sets.

def display_model_score(model, train, val, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

  val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
  print('Val loss: ', val_score[0])
  print('Val accuracy: ', val_score[1])
  print('-'*70)
  
  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

In [30]:
def equal_error_rate(y_true, y_pred):
    n_imp = tf.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

## Open embedding file

In [86]:
path = '/content/drive/MyDrive/ThermophilicESM1b'
suffix = '.pt'
delete = []
embeddings = []

In [181]:
proteins = []

In [182]:
# Basic Protocol 3 — Step 5
annotations = read_csv('Tannotations1024.csv')
#annotations = read_csv('annotationsUnique.csv')

In [177]:
annotations[:3]

,identifier,label,set
0,CYSO_AERPE,1,train
1,G1PDH_AERPE,1,train
2,PGMI_AERPE,1,train


In [183]:
for filename in annotations.identifier:
    file = os.path.join(path, filename + suffix)
    if (os.path.exists(file)):
      result = torch.load(file)
      rep = result.get('mean_representations')
      val = rep.values()
      val = list(val)
      val = np.array(val[0])
      val = np.reshape(val,(-1,1280))
      proteins.append((filename,val))
    else:
      delete.append(filename)

In [132]:
#proteins

In [184]:
# Basic Protocol 3 — Step 6
train_set = annotations[annotations.set == "train"]
test_set = annotations[annotations.set == "test"]

In [433]:
print(f"The train set contains {len(train_set)} samples, and we will test on {len(test_set)} samples.")

The train set contains 5091 samples, and we will test on 72 samples.


In [185]:
# Basic Protocol 3 — Step 7

training_identifiers = train_set.identifier.values
training_labels = train_set.label.values
print(len(training_identifiers))
print(len(training_labels))

2218
2218


In [186]:
testing_identifiers = test_set.identifier.values
testing_labels = test_set.label.values
print(len(testing_identifiers))
print(len(testing_labels))

549
549


In [187]:
seq = dict(proteins)
delete = list()

In [188]:
training_embeddings = list()
for identifier in training_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            training_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [189]:
testing_embeddings = list()
for identifier in testing_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            testing_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [190]:
np.where(training_identifiers=='RGYR2_AERPE')

(array([], dtype=int64),)

In [149]:
#training_identifiers = np.delete(training_identifiers,43)
#training_labels = np.delete(training_labels,43)

In [156]:
len(training_labels)

5091

In [157]:
len(training_embeddings)

5090

In [191]:
1024# A sanity check: make sure that the numbers are equal!
assert(len(training_identifiers) == len(training_embeddings))
assert(len(testing_identifiers) == len(testing_embeddings))

In [ ]:
len(training_embeddings)

In [192]:
delete

[]

## Training

In [151]:
arr_train = np.array(training_embeddings)
nsample, nx, ny = arr_train.shape
train_dataset = arr_train.reshape((nsample, nx*ny))
train_dataset.shape

(2907, 1280)

In [152]:
arr_test = np.array(testing_embeddings)
nsample, nx, ny = arr_test.shape
test_dataset = arr_test.reshape((nsample, nx*ny))
test_dataset.shape

(733, 1280)

### MLP

In [36]:
# Basic Protocol 3 — Step 8

multilayerperceptron = MLPClassifier(solver='lbfgs', random_state=10, max_iter=1000)

parameters = {
    'hidden_layer_sizes': [(30,), (1024,),(20,15)],
    'learning_rate_init': [0.001, 0.0001, 0.01],
    'solver':['lbfgs', 'adam','sgd'],
}

In [39]:
# Basic Protocol 3 — Step 9

classifiers = GridSearchCV(multilayerperceptron, parameters, cv=3, scoring="accuracy")
history = classifiers.fit(train_dataset, training_labels)
classifier = classifiers.best_estimator_

In [40]:
params = classifier.get_params()
params

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (1024,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.01,
 'max_fun': 15000,
 'max_iter': 1000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 10,
 'shuffle': True,
 'solver': 'sgd',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [69]:
import pandas as pd
from pandas import DataFrame

In [42]:
DataFrame(classifiers.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate_init,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,18.153142,4.924291,0.011807,0.000290,"(30,)",0.001,lbfgs,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.960519,0.949912,0.958726,0.956386,0.004636,22
1,9.694805,0.714433,0.010249,0.001404,"(30,)",0.001,adam,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.964054,0.954037,0.965802,0.961298,0.005184,10
2,16.165945,2.477267,0.012444,0.009543,"(30,)",0.001,sgd,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.950501,0.958161,0.972288,0.960317,0.009024,13
3,15.856573,4.155390,0.012849,0.001448,"(30,)",0.0001,lbfgs,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.960519,0.949912,0.958726,0.956386,0.004636,22
4,22.121074,1.216394,0.005636,0.000133,"(30,)",0.0001,adam,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.961697,0.958161,0.968750,0.962870,0.004402,7
5,31.387154,6.407714,0.006784,0.001834,"(30,)",0.0001,sgd,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.940483,0.945787,0.960495,0.948922,0.008465,27
6,16.920405,5.687503,0.011675,0.000225,"(30,)",0.01,lbfgs,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.960519,0.949912,0.958726,0.956386,0.004636,22
7,2.608895,0.384287,0.006436,0.001139,"(30,)",0.01,adam,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.964643,0.956983,0.964623,0.962083,0.003606,9
8,6.163116,0.927069,0.005607,0.000129,"(30,)",0.01,sgd,"{'hidden_layer_sizes': (30,), 'learning_rate_i...",0.958161,0.962286,0.971108,0.963852,0.005400,5
9,62.979339,10.395903,0.051303,0.001059,"(1024,)",0.001,lbfgs,"{'hidden_layer_sizes': (1024,), 'learning_rate...",0.962876,0.949912,0.961085,0.957957,0.005736,18


In [48]:
df =pd.DataFrame(classifiers.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/SOM/MLPESM1bMerged.xlsx')


In [49]:
# Basic Protocol 3 — Step 10

predicted_testing_labels = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

Our model has an accuracy of 0.96


In [ ]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
import numpy as np
        
bootstrap_performances = list()
performances = list()
Y = np.array(testing_labels) # convert list of groundtruths to numpy
Yhat = np.array(predicted_testing_labels) # same same for predictions
n_samples = len(Y) # get number of samples 
n_bootstrap = 1000 # number of bootstrap iterations
metric = matthews_corrcoef # the metric you want to compute
for i in range(n_bootstrap): # for each bootstrap draw
  subset = np.random.choice(n_samples, n_samples, replace=True) 
  # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
  bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
  performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
std_dev = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
sd = np.std(performances)
#print("Standard error after 1k bootstrapping iterations: {:.5f}".format(std_dev))|
#print(performances)
print(std_dev)
print(sd)

In [50]:
print(classification_report(testing_labels, predicted_testing_labels))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       647
           1       0.94      0.98      0.96       631

    accuracy                           0.96      1278
   macro avg       0.96      0.96      0.96      1278
weighted avg       0.96      0.96      0.96      1278



In [51]:
print(matthews_corrcoef(testing_labels, predicted_testing_labels))

0.9164055676333838


In [52]:
from pandas import DataFrame
cv_results = DataFrame(classifiers.cv_results_)

cv_results[['param_hidden_layer_sizes','split0_test_score', 'split1_test_score', 'split2_test_score']]

,param_hidden_layer_sizes,split0_test_score,split1_test_score,split2_test_score
0,"(30,)",0.960519,0.949912,0.958726
1,"(30,)",0.964054,0.954037,0.965802
2,"(30,)",0.950501,0.958161,0.972288
3,"(30,)",0.960519,0.949912,0.958726
4,"(30,)",0.961697,0.958161,0.968750
5,"(30,)",0.940483,0.945787,0.960495
6,"(30,)",0.960519,0.949912,0.958726
7,"(30,)",0.964643,0.956983,0.964623
8,"(30,)",0.958161,0.962286,0.971108
9,"(1024,)",0.962876,0.949912,0.961085


In [53]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [54]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)

In [55]:
confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [56]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

618 606 41 13
0.9793977812995246
0.9366306027820711


In [ ]:
#classifier.save_weights('/result/MLP.h5')

#fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = classifier.coefs_[0].min(), classifier.coefs_[0].max()
#for coef, ax in zip(classifier.coefs_[0].T, axes.ravel()):
#    ax.matshow(coef.reshape(1024, ), cmap=plt.cm.gray, vmin=0.5 * vmin, vmax=0.5 * vmax)
#    ax.set_xticks(())
#    ax.set_yticks(())

#plt.show()

In [ ]:
vmax

In [ ]:
weights = classifier.coefs_
print(len(weights))
print("shape",weights[0].shape,"content",weights[0])
print("shape",weights[1].shape,"content",weights[1])
#print("shape",weights[2].shape,"content",weights[2])


### LR

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

In [445]:
lr = LogisticRegression()
lr_history = lr.fit(train_dataset, training_labels)
lr.score(test_dataset,testing_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



0.875

In [446]:
scores = cross_val_score(lr, train_dataset, training_labels, scoring= 'f1', cv=5)
scores  

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/lin

array([0.96461072, 0.96780684, 0.96852792, 0.988     , 0.98105683])

In [447]:
scores = cross_validate(lr, train_dataset, training_labels, scoring= ['accuracy', 'precision', 'recall','f1'], cv=5)
scores

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/lin

{'fit_time': array([0.38805556, 0.3592608 , 0.35394788, 0.39808106, 0.39097023]),
 'score_time': array([0.00955415, 0.01525116, 0.00942445, 0.00964594, 0.01036739]),
 'test_accuracy': array([0.9656526 , 0.96856582, 0.96954813, 0.98821218, 0.98133595]),
 'test_precision': array([0.97546012, 0.97368421, 0.98350515, 0.988     , 0.97813121]),
 'test_recall': array([0.954, 0.962, 0.954, 0.988, 0.984]),
 'test_f1': array([0.96461072, 0.96780684, 0.96852792, 0.988     , 0.98105683])}

source https://www.kaggle.com/code/jnikhilsai/cross-validation-with-linear-regression

In [448]:
predicted_testing_labels = lr.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

Our model has an accuracy of 0.88


In [449]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
import numpy as np
        
bootstrap_performances = list()
performances = list()
f1_performances = list()
Y = np.array(testing_labels) # convert list of groundtruths to numpy
Yhat = np.array(predicted_testing_labels) # same same for predictions
n_samples = len(Y) # get number of samples 
n_bootstrap = 1000 # number of bootstrap iterations
metric = matthews_corrcoef # the metric you want to compute
for i in range(n_bootstrap): # for each bootstrap draw
  subset = np.random.choice(n_samples, n_samples, replace=True) 
  # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
  bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
  performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
  f1_performances.append(f1_score(testing_labels[subset], predicted_testing_labels[subset]))
std_dev = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
sd = np.std(performances)
sd_f1 = np.std(f1_performances)
#print("Standard error after 1k bootstrapping iterations: {:.5f}".format(std_dev))|
#print(performances)
print(std_dev)
print(sd)
print(sd_f1)

0.07924179419150848
0.0392763535023867
0.05618304525147778


In [450]:
print(classification_report(testing_labels, predicted_testing_labels))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90        42
           1       0.92      0.77      0.84        30

    accuracy                           0.88        72
   macro avg       0.89      0.86      0.87        72
weighted avg       0.88      0.88      0.87        72



In [451]:
print(matthews_corrcoef(testing_labels, predicted_testing_labels))

0.7446016458095223


In [452]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)

confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [453]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

23 40 2 7
0.7666666666666667
0.9523809523809523


In [454]:
#lr.save_weights('/result/LR.h5')
print(lr.coef_.shape)
lr_weights = lr.coef_
print(lr_weights)
print(np.max(lr_weights))
print(np.min(lr_weights))

(1, 1280)
[[ 0.09649021 -0.51717137 -0.56983673 ...  0.61797044  0.19880963
  -0.16163217]]
1.3667776545549295
-1.266978284575329


### CNN with GridSearch

In [153]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier

In [154]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from time import time

In [155]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [156]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [157]:
n_timesteps, n_features, n_outputs =train_dataset.shape[0], train_dataset.shape[1], Y_train.shape[1]

In [158]:
n_epochs = 30 # 30 
n_epochs_cv = 10 # 10  # reduce number of epochs for cross validation for performance reason

n_cv = 3
validation_ratio = 0.10

In [159]:
def create_cnn_model(pool_type='max', conv_activation='sigmoid', dropout_rate=0.10):
    # create model
    model = Sequential()
    
    # first layer: convolution
    #model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1))) 
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu',input_shape=(n_features,1)))
    # second series of layers: convolution, pooling, and dropout
    model.add(Conv1D(32, kernel_size=3, activation=conv_activation))  
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))     
    
    # third series of layers: convolution, pooling, and dropout    
    model.add(Conv1D(64, kernel_size=3, activation=conv_activation))   # 32   
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))     
      
    # fourth series
    model.add(Flatten())         
    model.add(Dense(64, activation='sigmoid')) # 64
    # add a dropout layer if rate is not null    
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate)) 
        
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile( 
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )    
    return model

In [160]:
cnn = create_cnn_model()

In [161]:
cnn.compile(
  optimizer='adam',
  loss='binary_crossentropy',  
  metrics=['accuracy'],
)

In [162]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 1278, 128)         512       
                                                                 
 conv1d_13 (Conv1D)          (None, 1276, 32)          12320     
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 638, 32)          0         
 1D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 638, 32)           0         
                                                                 
 conv1d_14 (Conv1D)          (None, 636, 64)           6208      
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 318, 64)          0         
 1D)                                                  

In [163]:
# optimize model 
start = time()

# create model
model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
# define parameters and values for grid search 
param_grid = {
    'pool_type': ['max', 'average'],
    'conv_activation': ['relu', 'tanh'],    
    'epochs': [10,30],
}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.



In [164]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=n_cv)
grid_result = grid.fit(train_dataset, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.bui

Epoch 1/30
91/91 [==============================] - 12s 121ms/step - loss: 0.3083 - accuracy: 0.8428
Epoch 2/30
91/91 [==============================] - 10s 105ms/step - loss: 0.1072 - accuracy: 0.9611
Epoch 3/30
91/91 [==============================] - 11s 123ms/step - loss: 0.0878 - accuracy: 0.9670
Epoch 4/30
91/91 [==============================] - 14s 157ms/step - loss: 0.0680 - accuracy: 0.9742
Epoch 5/30
91/91 [==============================] - 10s 106ms/step - loss: 0.0600 - accuracy: 0.9756
Epoch 6/30
91/91 [==============================] - 9s 101ms/step - loss: 0.0515 - accuracy: 0.9825
Epoch 7/30
91/91 [==============================] - 9s 101ms/step - loss: 0.0332 - accuracy: 0.9886
Epoch 8/30
91/91 [==============================] - 9s 104ms/step - loss: 0.0322 - accuracy: 0.9897
Epoch 9/30
91/91 [==============================] - 9s 101ms/step - loss: 0.0248 - accuracy: 0.9914
Epoch 10/30
91/91 [==============================] - 9s 101ms/step - loss: 0.0210 - accuracy: 0

In [165]:
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))   

In [166]:
# summarize results
print('time for grid search = {:.0f} sec'.format(time()-start))
display_cv_results(grid_result)

time for grid search = 1123 sec
Best score = 0.9398 using {'conv_activation': 'relu', 'epochs': 30, 'pool_type': 'max'}
mean test accuracy +/- std = 0.9212 +/- 0.0319 with: {'conv_activation': 'relu', 'epochs': 10, 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'relu', 'epochs': 10, 'pool_type': 'average'}
mean test accuracy +/- std = 0.9398 +/- 0.0193 with: {'conv_activation': 'relu', 'epochs': 30, 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'relu', 'epochs': 30, 'pool_type': 'average'}
mean test accuracy +/- std = 0.9106 +/- 0.0461 with: {'conv_activation': 'tanh', 'epochs': 10, 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'tanh', 'epochs': 10, 'pool_type': 'average'}
mean test accuracy +/- std = 0.9367 +/- 0.0230 with: {'conv_activation': 'tanh', 'epochs': 30, 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'tanh', 'epoch

In [167]:
DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_conv_activation,param_epochs,param_pool_type,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,267.159705,1.528563,3.409456,0.293209,relu,10,max,"{'conv_activation': 'relu', 'epochs': 10, 'poo...",0.889577,0.964912,0.909185,0.921225,0.031912,3
1,0.217552,0.003546,0.000000,0.000000,relu,10,average,"{'conv_activation': 'relu', 'epochs': 10, 'poo...",NaN,NaN,NaN,NaN,NaN,5
2,702.874873,1.109173,2.549119,0.205342,relu,30,max,"{'conv_activation': 'relu', 'epochs': 30, 'poo...",0.927761,0.966976,0.924665,0.939801,0.019258,1
3,0.051289,0.002311,0.000000,0.000000,relu,30,average,"{'conv_activation': 'relu', 'epochs': 30, 'poo...",NaN,NaN,NaN,NaN,NaN,6
4,259.972715,13.435674,4.365025,1.403421,tanh,10,max,"{'conv_activation': 'tanh', 'epochs': 10, 'poo...",0.916409,0.963880,0.851393,0.910561,0.046108,4
5,0.113137,0.043646,0.000000,0.000000,tanh,10,average,"{'conv_activation': 'tanh', 'epochs': 10, 'poo...",NaN,NaN,NaN,NaN,NaN,7
6,553.036613,0.390028,1.161626,0.255046,tanh,30,max,"{'conv_activation': 'tanh', 'epochs': 30, 'poo...",0.911249,0.966976,0.931889,0.936704,0.023004,2
7,0.070713,0.022356,0.000000,0.000000,tanh,30,average,"{'conv_activation': 'tanh', 'epochs': 30, 'poo...",NaN,NaN,NaN,NaN,NaN,8


In [168]:
df =pd.DataFrame(grid.cv_results_)
#new_path = '/content/test.xls'
#writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/SOM/CNNESM1bMerged.xlsx')

In [169]:
classifier = grid.best_estimator_

params = classifier.get_params()
params

{'verbose': 1,
 'conv_activation': 'relu',
 'epochs': 30,
 'pool_type': 'max',
 'build_fn': <function __main__.create_cnn_model(pool_type='max', conv_activation='sigmoid', dropout_rate=0.1)>}

In [170]:
# Basic Protocol 3 — Step 10

predicted_testing_labels = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

23/23 [==============================] - 1s 22ms/step
Our model has an accuracy of 0.97


In [171]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
import numpy as np
        
bootstrap_performances = list()
performances = list()
f1_performances = list()
Y = np.array(testing_labels) # convert list of groundtruths to numpy
Yhat = np.array(predicted_testing_labels) # same same for predictions
n_samples = len(Y) # get number of samples 
n_bootstrap = 1000 # number of bootstrap iterations
metric = matthews_corrcoef # the metric you want to compute
for i in range(n_bootstrap): # for each bootstrap draw
  subset = np.random.choice(n_samples, n_samples, replace=True) 
  # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
  bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
  performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
  f1_performances.append(f1_score(testing_labels[subset], predicted_testing_labels[subset]))
std_dev = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
sd = np.std(performances)
sd_f1= np.std(f1_performances)
#print("Standard error after 1k bootstrapping iterations: {:.5f}".format(std_dev))|
#print(performances)
print(std_dev)
print(sd)
print(sd_f1)

0.012272048945981152
0.006163318216779932
0.00637774572439919


In [172]:
print(classification_report(testing_labels, predicted_testing_labels))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       367
           1       0.98      0.96      0.97       366

    accuracy                           0.97       733
   macro avg       0.97      0.97      0.97       733
weighted avg       0.97      0.97      0.97       733



In [173]:
print(matthews_corrcoef(testing_labels, predicted_testing_labels))

0.9428716305435307


In [174]:
from pandas import DataFrame
cv_results = DataFrame(grid.cv_results_)

cv_results[['param_conv_activation','split0_test_score', 'split1_test_score', 'split2_test_score']]

,param_conv_activation,split0_test_score,split1_test_score,split2_test_score
0,relu,0.889577,0.964912,0.909185
1,relu,NaN,NaN,NaN
2,relu,0.927761,0.966976,0.924665
3,relu,NaN,NaN,NaN
4,tanh,0.916409,0.963880,0.851393
5,tanh,NaN,NaN,NaN
6,tanh,0.911249,0.966976,0.931889
7,tanh,NaN,NaN,NaN


In [175]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [176]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)

In [177]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [178]:
sn = TP / float(TP + FN)
print(sn)

0.9617486338797814


In [179]:
sp = TN / float(TN + FP)
print(sp)

0.9809264305177112


In [180]:
confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

### LR with GridSearch

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [95]:
lr = LogisticRegression()
lr.fit(train_dataset, training_labels) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [96]:
hyper_params = [{'n_features_to_select': list(range(1, 1024))}]
rfe = RFE(lr)

In [97]:
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = 3, 
                        verbose = 1,
                        return_train_score=True) 

In [98]:
model_cv.fit(train_dataset, training_labels)  

Fitting 3 folds for each of 1023 candidates, totalling 3069 fits


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


KeyboardInterrupt: ignored

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
df =pd.DataFrame(model_cv.cv_results_)
#new_path = '/content/test.xls'
#writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/SOM/LRESM1bMerged.xlsx')

In [ ]:
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
# final model
n_features_optimal = 10

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)             
rfe = rfe.fit(X_train, y_train)

# predict prices of X_test
y_pred = lm.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

### MLP

In [ ]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
#seed_value= 100
#tf.random.set_seed(seed_value)
n_cols = X_train.shape[1]
n_cols

In [ ]:
model = Sequential()
model.add(Dense(30,activation='relu', input_shape=(n_cols,)))
#model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss="mse", metrics="accuracy", )

In [ ]:
checkpointer = ModelCheckpoint(
    filepath='folder/.{epoch:03d}-{val_loss:.3f}.h5',
    verbose=1,
    save_best_only=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=150)

In [ ]:
history = model.fit(X_train, y_train,  
   epochs = 1000, 
   verbose = 0, 
   validation_data=(X_test,y_test), shuffle=True, validation_split = 0.1
  # callbacks = [es, checkpointer]
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.reshape(y_pred,(len(y_pred),)).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

In [ ]:
plot_history(history)

In [ ]:
display_model_score(model,
    [X_train, y_train],
    [X_test, y_test],
    [X_test, y_test],
    256)

In [ ]:
classes = np.unique(y_test)

confusion_matrix_data = confusion_matrix(y_test, y_pred, labels=classes)

confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
model.save_weights('MLP.h5')

### CNN

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import Flatten
from keras.regularizers import l2

In [ ]:
n_timesteps, n_features, n_outputs =train_dataset.shape[0], train_dataset.shape[1], Y_train.shape[1]

In [ ]:
# Building the CNN Model
cnn_model = Sequential()      # initializing the Sequential nature for CNN model

In [ ]:
cnn_model.add(Conv1D(filters=128, kernel_size=3, 
activation='relu',input_shape=(n_features,1)))
#cnn_model.add(Conv1D(filters=128, kernel_size=1, activation='relu'))
#cnn_model.add(Dropout(0.5))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(250, activation='relu'))
cnn_model.add(Dense(n_outputs, activation='sigmoid'))

In [ ]:
cnn_model.compile(loss='binary_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                  metrics=['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=150, verbose=1)

In [ ]:
# fit network
history = cnn_model.fit(train_dataset, Y_train, 
                        validation_data=(test_dataset, Y_test),
                        callbacks=[es],
                        epochs=500, batch_size=256, verbose=1
                        )

In [ ]:
plot_history(history)

In [ ]:
display_model_score(cnn_model,
    [train_dataset, Y_train],
    [test_dataset, Y_test],
    [test_dataset, Y_test],
    256)

In [ ]:
y_pred = cnn_model.predict(test_dataset)

In [ ]:
y_pred = np.reshape(y_pred,(len(y_pred),)).astype(int)

In [ ]:
print(classification_report(Y_test, y_pred))

In [ ]:
print(matthews_corrcoef(Y_test, y_pred))

In [ ]:
classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, y_pred, labels=classes)

confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
cnn_model.save_weights('CNN.h5')

## Visualization


In [ ]:
embeddings =list(map(lambda x: x[1], proteins))

In [ ]:
p = np.array(embeddings)
type(p)
p.shape
p = p.reshape(2809,1024)
p.shape

In [ ]:
def compute_pca(X, n_components=2):
    """
    Input:
        X: of dimension (m,n) where each row corresponds to a word vector
        n_components: Number of components you want to keep.
    Output:
        X_reduced: data transformed in 2 dims/columns + regenerated original data
    pass in: data as 2D NumPy array
    """

    ### START CODE HERE ###
    # mean center the data
    X_demeaned = X - X.mean(axis=0)

    # calculate the covariance matrix
    covariance_matrix = np.cov(X_demeaned, rowvar=False)

    # calculate eigenvectors & eigenvalues of the covariance matrix
    eigen_vals, eigen_vecs = np.linalg.eigh(covariance_matrix)

    # sort eigenvalue in increasing order (get the indices from the sort)
    idx_sorted = np.argsort(eigen_vals)
    
    # reverse the order so that it's from highest to lowest.
    idx_sorted_decreasing = list(reversed(idx_sorted))

    # sort the eigen values by idx_sorted_decreasing
    eigen_vals_sorted = eigen_vals[idx_sorted_decreasing]

    # sort eigenvectors using the idx_sorted_decreasing indices
    eigen_vecs_sorted = eigen_vecs[:, idx_sorted_decreasing]

    # select the first n eigenvectors (n is desired dimension
    # of rescaled data array, or dims_rescaled_data)
    eigen_vecs_subset = eigen_vecs_sorted[:, :n_components]

    # transform the data by multiplying the transpose of the eigenvectors with the transpose of the de-meaned data
    # Then take the transpose of that product.
    X_reduced = np.dot(eigen_vecs_subset.T, X_demeaned.T).T

    ### END CODE HERE ###

    return X_reduced

In [ ]:
result = compute_pca(p, 2)
plt.scatter(result[:, 0], result[:, 1])
plt.figure(figsize = (20, 16))
for i, word in enumerate(annotations.identifier):
    plt.annotate(word, xy=(result[i, 0] - 0.0005, result[i, 1] + 0.001))

plt.show()

In [ ]:
proteins_reduced = compute_pca(p, n_components=5)
print(proteins_reduced)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5, svd_solver='full')
pca.fit(p)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print(pca.score(p))
print(pca.transform(p))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
def reduce_dim(weights, components = 3, method = 'tsne'):
    """Reduce dimensions of embeddings"""
    if method == 'tsne':
        return TSNE(components, metric = 'cosine').fit_transform(weights)
    elif method == 'umap':
        # Might want to try different parameters for UMAP
        return UMAP(n_components=components, metric = 'cosine', 
                    init = 'random', n_neighbors = 5).fit_transform(weights)

In [ ]:
protein_r = reduce_dim(p, components = 2, method = 'tsne')
protein_r.shape

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

# Set shell to show all lines of output
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
InteractiveShell.ast_node_interactivity = 'last'

plt.figure(figsize = (20, 16))
plt.plot(protein_r[:, 0], protein_r[:, 1], 'r.')
plt.xlabel('TSNE 1'); plt.ylabel('TSNE 2'); plt.title('Protein Embeddings Visualized with TSNE');

In [ ]:
gen = ['non thermophilic', 'themophilic']
ints = annotations.label

plt.figure(figsize = (10, 8))

# Plot embedding
plt.scatter(protein_r[:, 0], protein_r[:, 1], c = ints)#, cmap = plt.cm.tab10)

# Add colorbar and appropriate labels
cbar = plt.colorbar()
cbar.set_ticks([])
for j, lab in enumerate(gen):
    cbar.ax.text(1, (2 * j + 1) / ((10) * 2), lab, ha='left', va='center')
cbar.ax.set_title('Label', loc = 'left')


plt.xlabel('TSNE 1'); plt.ylabel('TSNE 2'); plt.title('TSNE Visualization of Protein Embeddings');

### BLSTM

In [ ]:
from keras.layers import Embedding, Bidirectional, CuDNNLSTM, GlobalMaxPooling1D, LSTM
from keras.regularizers import l2

In [ ]:
x_input = Input(shape=(1024,))
emb = Embedding(20, 512, input_length=1024)(x_input)
bi_rnn = Bidirectional(CuDNNLSTM(256))(emb) #, kernel_regularizer=l2(0.0001), recurrent_regularizer=l2(0.0001), bias_regularizer=l2(0.0001)
#bi_rnn = LSTM(512)(emb)
#x = Dropout(0.2)(bi_rnn)

# sigmoid classifier
x_output = Dense(1, activation='sigmoid')(bi_rnn)

blstm = Model(inputs=x_input, outputs=x_output)

In [ ]:
blstm.summary()

In [ ]:
# Early Stopping
es = EarlyStopping(monitor='val_loss', patience=150, verbose=1)

In [ ]:
blstm.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )

In [ ]:
history = blstm.fit(
     X_train, y_train,
     epochs=100, batch_size=512,
     validation_data=(X_test, y_test),
     callbacks=[es]
     )

In [ ]:
plot_history(history)

In [ ]:
display_model_score(blstm,
    [X_train, y_train],
    [X_test, y_test],
    [X_test, y_test],
    256)

In [ ]:
y_pred = blstm.predict(X_test)

In [ ]:
y_pred = np.reshape(y_pred,(len(y_pred),)).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
blstm.save_weights('BLSTM.h5')